In [ ]:
!pip3 install pandas dash

In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import wget

In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_launch_dash = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

In [5]:
# Read the airline data into pandas dataframe

spacex_df=pd.read_csv(spacex_launch_dash,index_col=0)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [6]:
Launch_sites=spacex_df.groupby('Launch Site')['class'].value_counts()
Launch_sites

Launch Site   class
CCAFS LC-40   0        19
              1         7
CCAFS SLC-40  0         4
              1         3
KSC LC-39A    1        10
              0         3
VAFB SLC-4E   0         6
              1         4
Name: class, dtype: int64

In [7]:
filtered_df=spacex_df.groupby('Launch Site')['class'].value_counts().reset_index(name='value counts')
filtered_df

,Launch Site,class,value counts
0,CCAFS LC-40,0,19
1,CCAFS LC-40,1,7
2,CCAFS SLC-40,0,4
3,CCAFS SLC-40,1,3
4,KSC LC-39A,1,10
5,KSC LC-39A,0,3
6,VAFB SLC-4E,0,6
7,VAFB SLC-4E,1,4


In [8]:
filtered_df=spacex_df
filtered_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
filtered_df

,Launch Site,class,class count
0,CCAFS LC-40,0,19
1,CCAFS LC-40,1,7
2,CCAFS SLC-40,0,4
3,CCAFS SLC-40,1,3
4,KSC LC-39A,0,3
5,KSC LC-39A,1,10
6,VAFB SLC-4E,0,6
7,VAFB SLC-4E,1,4


In [9]:
success=spacex_df.groupby('Launch Site')['class'].mean()
success

Launch Site
CCAFS LC-40     0.269231
CCAFS SLC-40    0.428571
KSC LC-39A      0.769231
VAFB SLC-4E     0.400000
Name: class, dtype: float64

In [10]:
success=spacex_df.groupby('Launch Site')['class'].mean().reset_index()
success

,Launch Site,class
0,CCAFS LC-40,0.269231
1,CCAFS SLC-40,0.428571
2,KSC LC-39A,0.769231
3,VAFB SLC-4E,0.400000


In [11]:
# Create a dash application
app = dash.Dash(__name__)
# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

In [ ]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                                       ],
                                             value='All',
                                             placeholder="Select the Lunch Site",
                                             searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',min=0, max=10000, step=1000,marks={0: '0',100: '100'},value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Success Rate')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df=spacex_df[spacex_df['Launch Site']== entered_site]
        filtered_df=filtered_df.groupby('Launch Site')['class'].value_counts().reset_index(name='value counts')
        fig = px.pie(filtered_df, values='value counts', names='class', title=entered_site)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])
def scatter(entered_site,payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    
    if entered_site=='ALL':
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success rate for all sites')
        return fig
    else:
        fig=px.scatter(filtered_df[filtered_df['Launch Site']==entered_site],x='Payload Mass (kg)',y='class',color='Booster Version Category',title=f"Success rate for site {entered_site}")
        return fig
# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Oct/2022 04:08:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2022 04:08:16] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_6_2m1665367999.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2022 04:08:16] "GET /_dash-component-suites/dash/deps/react@16.v2_6_2m1665367999.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2022 04:08:16] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_6_2m1665367999.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2022 04:08:16] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_6_2m1665367999.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2022 04:08:16] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_6_2m1665367998.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2022 04:08:16] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_6_2m1665367999.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2022 04:08:16] "GET /_dash-component-suites/d